This is a note taken down while attending the course "Serverless LLM Apps Amazon Bedrock" tutorial from DeepLearning.AI. 

## Setup bedrock env and simple prompt

```
import boto3
import json

bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

prompt = "Write a one sentence summary of Las Vegas."

kwargs = {
    "modelId": "amazon.titan-text-lite-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
```

**print response**

```
{'ResponseMetadata': {'RequestId': '2e0d3c1d-b113-4414-8e37-e0660de2d5eb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 22 Aug 2024 01:31:08 GMT',
   'content-type': 'application/json',
   'content-length': '196',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2e0d3c1d-b113-4414-8e37-e0660de2d5eb',
   'x-amzn-bedrock-invocation-latency': '848',
   'x-amzn-bedrock-output-token-count': '19',
   'x-amzn-bedrock-input-token-count': '9'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x7f12aa9cc760>}
 
 ```
**print body**
```
response_body = json.loads(response.get('body').read())
print(json.dumps(response_body, indent=4))

{
    "inputTextTokenCount": 9,
    "results": [
        {
            "tokenCount": 19,
            "outputText": "\nLas Vegas is a popular tourist destination known for its casinos, nightlife, and entertainment.",
            "completionReason": "FINISH"
        }
    ]
}
```
**print output text**
```
print(response_body['results'][0]['outputText'])

Las Vegas is a popular tourist destination known for its casinos, nightlife, and entertainment.

```


## Configuring prompts

```
kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body" : json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 100,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}
```

## Summarize audio transcript
```
with open('transcript.txt', "r") as file:
    dialogue_text = file.read()

prompt = f"""The text between the <transcript> XML tags is a transcript of a conversation. 
Write a short summary of the conversation.

<transcript>
{dialogue_text}
</transcript>

Here is a summary of the conversation in the transcript:"""

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)

response_body = json.loads(response.get('body').read())
generation = response_body['results'][0]['outputText']

```

In [2]:
#read audio
from IPython.display import Audio
audio = Audio(filename="output.mp3")
display(audio)

### Add logs

Set permissions in IAM for log group then add configs to setup logs in AWS

```
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

log_group_name = '/my/amazon/bedrock/logs'

cloudwatch.create_log_group(log_group_name)


loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)
```


Can also use Lambda functions to implement this code - which is quickest way to implement audio summarizer